In [1]:
!dir data

 驱动器 C 中的卷没有标签。
 卷的序列号是 0006-FDC5

 C:\Users\dell\dockerdata\乘用车细分市场销量预测\data 的目录

2019/09/11  14:12    <DIR>          .
2019/09/11  14:12    <DIR>          ..
2019/09/11  13:30           237,991 evaluation_public.csv
2019/09/11  13:31            51,712 submit_example.csv
2019/08/23  17:36           369,997 Train.zip
2019/09/12  15:33         1,545,277 train_sales_data.csv
2019/09/12  15:33         1,399,503 train_search_data.csv
2019/09/12  15:33            47,975 train_user_reply_data.csv
               6 个文件      3,652,455 字节
               2 个目录 49,158,709,248 可用字节


In [2]:
import numpy as np
import pandas as pd

In [3]:
train_sales_data=pd.read_csv('data/train_sales_data.csv')
train_sales_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466
2,内蒙古,150000,3c974920a76ac9c1,SUV,2016,1,257
3,北京,110000,3c974920a76ac9c1,SUV,2016,1,408
4,四川,510000,3c974920a76ac9c1,SUV,2016,1,610


In [28]:
train_sales_data[train_sales_data['model']=='02aab221aabc03b9'].head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
506,上海,310000,02aab221aabc03b9,Sedan,2016,1,617
507,云南,530000,02aab221aabc03b9,Sedan,2016,1,238
508,内蒙古,150000,02aab221aabc03b9,Sedan,2016,1,92
509,北京,110000,02aab221aabc03b9,Sedan,2016,1,799
510,四川,510000,02aab221aabc03b9,Sedan,2016,1,559


In [29]:
lsindex=(train_sales_data['model']=='02aab221aabc03b9')&(train_sales_data['province']=='上海')&\
            (train_sales_data['regMonth']==2)
train_sales_data[lsindex]

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
1826,上海,310000,02aab221aabc03b9,Sedan,2016,2,279
17666,上海,310000,02aab221aabc03b9,Sedan,2017,2,494


In [17]:
cols=['province','adcode','model','bodyType','regMonth']
train_sales_data_pivot=train_sales_data.pivot_table(values='salesVolume',index=cols,columns='regYear',aggfunc='sum')

In [18]:
stats=['province','adcode','model','bodyType','regMonth','predictsales']
train_sales_data_pivot.head()

regYear                                             2016  2017
province adcode model            bodyType regMonth            
上海       310000 02aab221aabc03b9 Sedan    1          617   679
                                          2          279   494
                                          3          554   731
                                          4          507   600
                                          5          485   672

In [37]:
P=0.4
predict_data=(P*train_sales_data_pivot.loc[:,2016]+(1-P)*train_sales_data_pivot.loc[:,2017]).map(np.int).reset_index()
predict_data.columns=stats=['province','adcode','model','bodyType','regMonth','meanVolum']
predict_data.head()

,province,adcode,model,bodyType,regMonth,meanVolum
0,上海,310000,02aab221aabc03b9,Sedan,1,654
1,上海,310000,02aab221aabc03b9,Sedan,2,408
2,上海,310000,02aab221aabc03b9,Sedan,3,660
3,上海,310000,02aab221aabc03b9,Sedan,4,562
4,上海,310000,02aab221aabc03b9,Sedan,5,597


In [36]:
evaluation_public_data=pd.read_csv('data/evaluation_public.csv')
evaluation_public_data.head()

,id,province,adcode,model,regYear,regMonth,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,NaN
1,2,云南,530000,3c974920a76ac9c1,2018,1,NaN
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,NaN
3,4,北京,110000,3c974920a76ac9c1,2018,1,NaN
4,5,四川,510000,3c974920a76ac9c1,2018,1,NaN


In [38]:
predicting_data=pd.merge(evaluation_public_data,predict_data,on=['province','adcode','model','regMonth'],how='left')

In [41]:
predicting_data['forecastVolum']=predicting_data['meanVolum']
predicting_data[['id','forecastVolum']].to_csv('data/mean_predicting_data.csv',index=False)